In [73]:
from collections import defaultdict 
import json
import jsonpickle 

#Node structure for graph
class Node:

    def __init__(self,src,dest,wt):
        self.src = src
        self.dest = dest
        self.wt = wt


#This class represents an un-directed graph using adjacency list representation 
class Graph: 
   
    def __init__(self,vertices): 
        self.V = vertices #No. of vertices 
        self.V_org = vertices 
        self.graph = defaultdict(list) # default dictionary to store graph 
   
    # function to add an edge to graph 
    def addEdge(self,u,v,w): 
        self.graph[u].append(Node(u,v,w))
        self.graph[v].append(Node(v,u,w))

    #function to print graph
    def printGraph(self):
        for i in self.graph:
            print(str(i) + " is connected to ")
            for node in self.graph[i]:
                print(str(node.dest) + "(Weight = " + str(node.wt) + ")" + " ")
            print("\n")

    
    def BFSi(self, s, max_levels):
        visited = set()
 
        queue = []
 
        queue.append(s)
        visited.add(s)
        level = 0
        result = {}
        while queue:
            aux = []
            result[level] = []
            
            while queue:
                s = queue.pop(0)
                visited.add(s)
                result[level].append(s)
                for node in self.graph[s]:
                    if node.dest not in visited:
                        aux.append(node.dest)
                        visited.add(node.dest)
            level += 1
            if level > max_levels:
                break
            for node in aux:
                queue.append(node)
            
        return result            
    
    def export_network(self, filename = "output"):
        filename += ".json"
        obj = jsonpickle.encode(self.graph)
        with open(filename, "w") as outfile: 
            json.dump(obj, outfile)

    def import_network(self, filename = "output"):
        filename += ".json"
        with open('output.json') as json_file:
            data = json.load(json_file)
            self.graph = jsonpickle.decode(data)
            self.V = len(self.graph)
            self.V_org = len(self.graph)

In [74]:
# Create a graph SAMPLE use 
g = Graph(4) 
g.addEdge(0, 1, 2) 
g.addEdge(0, 2, 2) 
g.addEdge(1, 2, 1) 
g.addEdge(1, 3, 1) 
g.addEdge(2, 3, 2) 
# g.printGraph()
print(g.BFSi(3,3))

g = Graph(4) 
g.addEdge('a', 'b', 2) 
g.addEdge('a', 'c', 2) 
g.addEdge('b', 'c', 1) 
g.addEdge('b', 'd', 1) 
g.addEdge('c', 'd', 2) 
# g.printGraph()
print(g.BFSi('d',3))

{0: [3], 1: [1, 2], 2: [0]}
{0: ['d'], 1: ['b', 'c'], 2: ['a']}


In [75]:
g.export_network()

In [76]:
print(len(g.graph))
print(g.V)

4
4


In [77]:
gg = Graph(100)
gg.import_network()
print(gg.graph)
print(gg.BFSi('d',3))

defaultdict(<class 'list'>, {'a': [<__main__.Node object at 0x7f5f6030e850>, <__main__.Node object at 0x7f5f6030e450>], 'b': [<__main__.Node object at 0x7f5f6030eed0>, <__main__.Node object at 0x7f5f6030edd0>, <__main__.Node object at 0x7f5f6030ea10>], 'c': [<__main__.Node object at 0x7f5f6030ed50>, <__main__.Node object at 0x7f5f6030e490>, <__main__.Node object at 0x7f5f6030ef10>], 'd': [<__main__.Node object at 0x7f5f6030e890>, <__main__.Node object at 0x7f5f6030eb10>]})
{0: ['d'], 1: ['b', 'c'], 2: ['a']}


In [78]:
print(len(gg.graph))
print(gg.V)

4
4
